# Example: Working with models from CLI

HydroMT has the following high-level functionality from the Command Line Interface (CLI) for setting up models from raw data or adjusting models:

- **building** a model: building a model from scratch.
- **updating** a model: adding or changing model components of an existing model.

Here we show how to build and update a hypothetical gridded model that contains a ``grid`` and a ``config`` from the command line interface (CLI) based on the generic HydroMT **example_model**.

Lets first check which models are available in our environment:

In [ ]:
!hydromt --models

## Build a model from CLI

To build a model you always follow the next four steps.

1. Prepare or use a pre-defined data catalog with all the required data sources
2. Define your model region, see the overview of [model region options](https://deltares.github.io/hydromt/latest/user_guide/models/model_region.html).
3. Prepare a [model workflow file](https://deltares.github.io/hydromt/latest/user_guide/models/model_workflow.html) which describes the complete pipeline to build your model.
4. Build you model using the CLI or Python interface

Here we focus on steps 2-4 and use data from the [predefined artifact_data catalog](https://deltares.github.io/hydromt/latest/user_guide/data_catalog/data_existing_cat.html#available-pre-defined-data-catalogs).

Using the **hydromt build** method we can setup a complete model from scratch. Let’s get an overview of the method and its arguments.

> Note the required MODEL (i.e. name of the model) and MODEL_ROOT (i.e. folder where to save the model) arguments.

In [ ]:
!hydromt build --help

As a first step, we want to create a grid for our example model. We will do this by calling the ``example_model.grid.create_from_region`` method. We will based our grid from a bounding box with a resolution of 0.05 degree in EPSG 4326.

For this we will create the following workflow file below, where we define the region for our grid.

> The %%writefile magic saves the content below to a file. The content of the .yaml file start from the second line.

In [ ]:
%%writefile tmp_build_example_model_min.yml

steps:
  - grid.create_from_region:
      region:
        bbox: [11.70, 45.35, 12.95, 46.70]
      res: 0.05
      crs: 4326

Now let's call the HydroMT ``build`` command with our newly created **build_example_model.yml** workflow file and using the **artifact_data** catalog:

In [ ]:
!hydromt build example_model ./tmp_example_model_min -i tmp_build_example_model_min.yml -d artifact_data --fo -v

The example above means the following: run **hydromt build** with:

- ``example_model``: i.e. build a generic GridModel instance
- ``./tmp_example_model_min``: output model folder
- ``-i tmp_build example_model_min.yml``: path to the model build workflow file
- ``-d artifact_data``: data catalog to use
- ``-vv`` : give some extra verbosity (1 * v) to display feedback on screen. Now INFO messages are provided.

Let's see which files where created for our model:

In [ ]:
# print MODEL_ROOT folder
import os


def print_dir(root):
    for path, _, files in os.walk(root):
        print(path)
        for name in files:
            if name.endswith(".xml"):
                continue
            print(f" - {name}")


print_dir("tmp_example_model_min")

We can see that in this case, not many files were created, only the grid and the hydromt log file.

So now let's add more steps to our workflow file to also add a configuration file with some settings, re-create our grid and populate it with some data (e.g. elevation and landuse).
We will use the following workflow file:

In [ ]:
%%writefile tmp_build_example_model.yml

steps:
  - config.update:
      data:
        header.settings: value
        timers.end: "2010-02-15"
        timers.start: "2010-02-05"
  - grid.create_from_region:
      region:
        bbox: [11.70, 45.35, 12.95, 46.70]
      res: 0.05
      crs: 4326
  - grid.add_data_from_rasterdataset:
      raster_data: merit_hydro_ihu
      variables:
        - elevtn
        - basins
      reproject_method:
        - average
        - mode
  - grid.add_data_from_rasterdataset:
      raster_data: vito_2015
      fill_method: nearest
      reproject_method: mode
      rename:
        vito: landuse
  - write:
      components:
        - grid
        - config

In this configuration, you see that we will prepare quite a lot of data for our grid model using some of the generic model methods for grid. We will prepare:

- Update some config options using [config.update](https://deltares.github.io/hydromt/latest/_generated/hydromt.model.components.ConfigComponent.update.html#hydromt.model.components.ConfigComponent.update)
- Create a regular grid from a bounding box region using [grid.create_from_region](https://github.com/Deltares/hydromt/blob/fbb17de5046e657a6257959c5ee48634eab5adab/hydromt/model/example/example_grid_component.py#L58)
- A couple of grid based on reprojection of MERIT Hydro IHU (elevation, basins) and VITO (landuse) using [grid.add_data_from_rasterdataset](https://github.com/Deltares/hydromt/blob/fbb17de5046e657a6257959c5ee48634eab5adab/hydromt/model/example/example_grid_component.py#L156). Note that you can use the same several times in the same workflow file.

Let's now build this model:

In [ ]:
!hydromt build example_model ./tmp_example_model -i tmp_build_example_model.yml -d artifact_data --fo -v

The example above means the following: run **hydromt build** with:

- ``example_model``: i.e. build a generic GridModel instance
- ``./tmp_example_model``: output model folder
- ``-i tmp_build_example_model.yaml``: use this .yaml file to configure the model build
- ``-d artifact_data``: parse the pre-defined artifact_data
- ``-vv``: give some extra verbosity (1 * v) to display feedback on screen. Now INFO messages are provided.

Let’s check some of the outputs that were produced:

In [ ]:
# Files created
print_dir("tmp_example_model")

In [ ]:
# checkout the content of the hypothetical model simulation configuration
fn_yaml = "tmp_example_model/settings.toml"
with open(fn_yaml, "r") as f:
    txt = f.read()
print(txt)

In [ ]:
# List of variables in grid.nc
from hydromt.model import ExampleModel

model = ExampleModel(root="./tmp_example_model", mode="r")
ds = model.grid.data

print(f"Variables available in grid.nc: {list(ds.data_vars)}")

In [ ]:
# Plot one of the variable (change the name below to plot a different variable)
var = "elevtn"
ds[var].plot()

## Update a model from CLI

Using the **hydromt update** method we can update an existing model with new components or modify existing components. Let’s get an overview of the method and its arguments.

Note that the ``MODEL`` (i.e. name of the model), and ``MODEL_ROOT`` (i.e. folder of existing model) are still required. There is an optional ``-o`` ``--model-out`` option to save the updated model in a different directory.

In [ ]:
!hydromt update --help

In this basic example we use the **hydromt update** method to update the ExampleModel instance with an upstream area raster map. Then we write only the updated model map component to file.

In [ ]:
%%writefile tmp_update_example_model.yml

steps:
  - grid.add_data_from_rasterdataset:
      raster_data: merit_hydro_ihu
      variables:
        - uparea
      reproject_method:
        - max
  - grid.write:

In [ ]:
!hydromt update example_model ./tmp_example_model -o ./tmp_example_model_update -i tmp_update_example_model.yml -d artifact_data --fo -v

The example above means the following: run **hydromt update** with:

- ``example_model``: i.e. update a ExampleModel instance
- ``./tmp_example_model``: the folder of the to-be updated model
- ``-o ./tmp_example_model_update``: the folder of the updated model
- ``-i tmp_update_example_model.yml``: the hydromt workflow file listing the methods to be executed
- ``-d artifact_data``: use the pre-defined artifact_data catalog
- ``-v``: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

Let's check the content of the updated model and that we should now have an uparea variable available in the updated model grid:

In [ ]:
# note the difference with the original model
print_dir("tmp_example_model_update")

In [ ]:
# List of variables in grid.nc
from hydromt.model import ExampleModel

model = ExampleModel(root="./tmp_example_model_update", mode="r")
ds = model.grid.data

print(f"Variables available in grid.nc: {list(ds.data_vars)}")